# im2colとcol2imの実装

In [ ]:
import numpy as np

## im2colの実装

### [演習]
* 以下のim2col関数を完成させましょう

In [ ]:
# ヒント
print("5//2 = ", 5//2)
print()

input_data = np.random.randn(1,2,2,2).round(2)
print("input_data=\n",input_data)
pad = 2
print()
print("パディング=")
print(np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant'))
print()
print("軸の入れ替え")
input_data.transpose(0, 1,3,2)

In [ ]:
def im2col(input_data, filter_h, filter_w, stride=1, pad=0, constant_values=0):
    """
    input_data : (データ数, チャンネル数, 高さ, 幅)の4次元配列からなる入力データ. 画像データの形式を想定している
    filter_h : フィルターの高さ
    filter_w : フィルターの幅
    stride : ストライド数
    pad : パディングサイズ
    constant_values : パディング処理で埋める際の値
    return : 2次元配列
    """
    
    # 入力画像(元画像)のデータ数, チャンネル数, 高さ, 幅を取得する
    N, C, H, W = input_data.shape 
    
    # 出力(畳み込みまたはプーリングの演算後)の形状を計算する
    out_h =        # 出力画像の高さ(端数は切り捨てる)                                                                                                              # <- 穴埋め
    out_w =       # 出力画像の幅(端数は切り捨てる)                                                                                                                  # <- 穴埋め

    # パディング処理
    img = np.pad(input_data, [                                ],                                                # <- 穴埋め
                  'constant', constant_values=constant_values) # pad=1以上の場合、周囲を0で埋める
    
    # 配列の初期化
    col = np.zeros((N, C,     ,     , out_h, out_w))                                                            # <- 穴埋め

    # 配列を並び替える(フィルター内のある1要素に対応する画像中の画素を取り出してcolに代入する)
    for y in range(   ):                                                                                         # <- 穴埋め
        """
        フィルターの高さ方向のループ
        """
        y_max = y + stride*out_h
        
        for x in range(   ):                                                                                    # <- 穴埋め
            """
            フィルターの幅方向のループ
            """
            x_max = x + stride*out_w
            
            # imgから値を取り出し、colに入れる
            col[:, :, y, x, :, :] = img[                                        ]                              # <- 穴埋め
            # y:y_max:strideの意味  :  yからy_maxまでの場所をstride刻みで指定している
            # x:x_max:stride の意味  :  xからx_maxまでの場所をstride刻みで指定している

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1) # 軸を入れ替えて、2次元配列(行列)に変形する
    return col

In [ ]:
# データ数が1の場合の確認
np.random.seed(1234)
x1 = np.random.randn(1,3,3,3)
print("x1=",x1.round(2))
print()
print("パディングなし")
col1 = im2col(x1, 2, 2, stride=1, pad=0)
print("col1=",col1.round(2))
print()
print("パディングあり")
col1_p = im2col(x1, 2, 2, stride=1, pad=1)
print("col1_p=",col1_p.round(2))

In [ ]:
# データ数が5の場合の確認
np.random.seed(1234)
x5 = np.random.randn(5,3,3,3)
print("x5=",x5.round(2))
print()
print("パディングなし")
col5 = im2col(x5, 2, 2, stride=1, pad=0)
print("col5=",col5.round(2))
print()
print("パディングあり")
col5_p = im2col(x5, 2, 2, stride=1, pad=1)
print("col5_p=",col5_p.round(2))

## col2imの実装

### [演習]
* 以下のcol2im関数を完成させましょう

In [ ]:
def col2im(col, input_shape, filter_h, filter_w, stride=1, pad=0, is_backward=False):
    """
    Parameters
    ----------
    col : 2次元配列
    input_shape : 入力データの形状（例：(10, 1, 28, 28)）
    filter_h : フィルターの高さ
    filter_w : フィルターの幅
    stride : ストライド数
    pad : パディングサイズ
    return : (データ数, チャンネル数, 高さ, 幅)の配列
    -------
    """
    
    # 入力画像(元画像)のデータ数, チャンネル数, 高さ, 幅を取得する
    N, C, H, W = input_shape
    
    # 出力(畳み込みまたはプーリングの演算後)の形状を計算する
    out_h =            # 出力画像の高さ(端数は切り捨てる)                                                                                                                  # <- 穴埋め
    out_w =          # 出力画像の幅(端数は切り捨てる)                                                                                                                         # <- 穴埋め
    
    # 配列の形を変えて、軸を入れ替える
    col = col.reshape(N, out_h, out_w, C, filter_h, filter_w).transpose(0, 3, 4, 5, 1, 2)

    # 配列の初期化
    img = np.zeros((   ,    , H + 2*pad + stride - 1, W + 2*pad + stride - 1))                                       # <- 穴埋め
    # 2*padは、pad分を大きくとっている。
    # (stride - 1)は、im2colで画像が切り捨てられる場合のサイズ調整分。
    # im2colで、strideを2以上に設定した場合、あるひとつの方向に最大で(stride - 1)個の画素が切り捨てられる。
    # このような切り捨てが発生する場合、col2imのimg[:, :, y:y_max:stride, x:x_max:stride] でスライスを使って
    # stride刻みで値を代入する際にエラーを出して止まってしまう。そのため、縦横ともに(stride - 1)個分だけ余分に配列を確保しておき、
    # 最後に余分なところを切り捨てることでサイズを調整している。
    
    # 配列を並び替える
    for y in range(   ):                                                                                              # <- 穴埋め
        """
        フィルターの高さ方向のループ
        """        
        y_max = y + stride*out_h
        
        for x in range(    ):                                                                                          # <- 穴埋め
            """
            フィルターの幅方向のループ
            """            
            x_max = x + stride*out_w
            
            if is_backward:
                """
                逆伝播計算の場合
                """
                img[:, :, y:y_max:stride, x:x_max:stride] += col[             ] # 伝わってきた勾配を足していく                  # <- 穴埋め
            else:
                """
                元のimに戻ることを確認したい場合
                """
                img[:, :, y:y_max:stride, x:x_max:stride] = col[               ]                                      # <- 穴埋め

    return img[:, :,       :H +    ,     :W +    ] # pad分は除いておく(pad分を除いて真ん中だけを取り出す)                            # <- 穴埋め

In [ ]:
# データ数が1の場合の確認
im1= col2im(col1, input_shape=(1, 3, 3, 3), filter_h=2, filter_w=2, stride=1, pad=0)
print("im1=", im1.round(2))
print()
print("元の形に戻っていることを確認(値が0であればok)")
print((x1 - im1 ).sum())
print()
im1_p= col2im(col1_p, input_shape=(1, 3, 3, 3), filter_h=2, filter_w=2, stride=1, pad=1)
print("im1_p=", im1_p.round(2))
print()
print("元の形に戻っていることを確認(値が0であればok)")
print((x1 - im1_p ).sum())

In [ ]:
# データ数が5の場合の確認
im5 = col2im(col5, input_shape=(5, 3, 3, 3), filter_h=2, filter_w=2, stride=1, pad=0)
print("im5=", im5.round(2))
print()
print("元の形に戻っていることを確認(値が0であればok)")
print((x5 - im5 ).sum())
print()

im5_p = col2im(col5_p, input_shape=(5, 3, 3, 3), filter_h=2, filter_w=2, stride=1, pad=1)
print("im5_p=", im5_p.round(2))
print()
print("元の形に戻っていることを確認(値が0であればok)")
print((x5 - im5_p ).sum())
print()